In [ ]:
import pandas as pd
import numpy as np

## Load Data

**Load metadata**

In [ ]:
df_artworks = pd.read_csv('/root/work/datasets/train_mayors_style_encoded_with_url.csv')
df_artworks.head()

In [ ]:
df_artworks.shape

**Load matrix**

In [ ]:
artwork_code_matrix = np.load('/root/work/datasets/train_mayors_style_w_encoded.npy')
artwork_code_matrix.shape

### Get Impressionist artworks

In [ ]:
df_impressionist = df_artworks[df_artworks['style'] == 'Impressionism']
df_impressionist.head()

In [ ]:
df_impressionist.shape

**NaN values**

In [ ]:
df_impressionist['date'].isnull().value_counts()

In [ ]:
df_impressionist = df_impressionist.dropna(subset=['date'])
df_impressionist.head()

**Normalize artist name**

In [ ]:
import unicodedata

def normalize_title(title):
    return unicodedata.normalize('NFKD', title.lower()).encode('ASCII', 'ignore').decode('utf8')

In [ ]:
df_impressionist['artist'] = df_impressionist['artist'].apply(normalize_title)

**Normalize date**

In [ ]:
dates = df_impressionist['date']
dates.head()

In [ ]:
dates_normalize = dates.str.extract(r'(?P<begining>.+)?(?P<date>\d\d\d\d)(?P<end>.+)?')
dates_normalize.head()

In [ ]:
df_impressionist['date'] = dates_normalize['date']
df_impressionist.head()

### Sort values

In [ ]:
df_impressionist_sorted = df_impressionist.sort_values(by=['date', 'artist'], ascending=True)
df_impressionist_sorted.head()

### Get artwork's code

In [ ]:
matrix_sorted = artwork_code_matrix[df_impressionist_sorted.index]
matrix_sorted.shape

In [ ]:
matrix_sorted[0]

### Save data

In [ ]:
df_impressionist_sorted.to_csv('impressionsist_sorted.csv', index=False)

In [ ]:
np.save('impressionist_sorted_matrix',matrix_sorted)